## Efficientnet

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# 시드 고정
seed = 2021
deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

if deterministic:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, test_loss):
        if self.best_loss is None or test_loss < self.best_loss - self.delta:
            self.best_loss = test_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# 하이퍼파라미터 설정
model_name = 'efficientnet-b0'
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001
image_size = EfficientNet.get_image_size(model_name)

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터셋 분할 (80% 훈련, 20% 테스트)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_pretrained(model_name, num_classes=num_classes).to(device)
print("Using device:", device)

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100
early_stopping = EarlyStopping(patience=10)

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "./best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0
Using device: cuda

[Epoch 1/100]


Train Loss: 0.9612 | Train Acc: 0.6935 | Test Loss: 1.8026 | Test Acc: 0.5706
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.3584 | Train Acc: 0.8884 | Test Loss: 1.3889 | Test Acc: 0.7175
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.1668 | Train Acc: 0.9534 | Test Loss: 1.1210 | Test Acc: 0.7740
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.1748 | Train Acc: 0.9534 | Test Loss: 0.9343 | Test Acc: 0.8305
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.0623 | Train Acc: 0.9831 | Test Loss: 0.9367 | Test Acc: 0.8588

[Epoch 12/100]


Train Loss: 0.1059 | Train Acc: 0.9718 | Test Loss: 0.8543 | Test Acc: 0.8418

[Epoch 13/100]


Train Loss: 0.1978 | Train Acc: 0.9350 | Test Loss: 0.6111 | Test Acc: 0.8757

[Epoch 14/100]


Train Loss: 0.1272 | Train Acc: 0.9562 | Test Loss: 0.6575 | Test Acc: 0.8701

[Epoch 15/100]


Train Loss: 0.1561 | Train Acc: 0.9463 | Test Loss: 0.6552 | Test Acc: 0.8757
Early Stopping Triggered!

Best Accuracy: 0.8757


## Vgg19

In [15]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# ✅ 시드 고정
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ 하이퍼파라미터 설정
num_classes = 6
batch_size = 32
epochs = 50
learning_rate = 0.001  # VGG19의 안정적인 학습을 위해 낮은 lr 사용
image_size = 224  # VGG19의 입력 크기

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ 데이터셋 로드
dataset_path = 'C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# ✅ 데이터셋 분할 (80% 훈련, 20% 테스트)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# ✅ DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ✅ VGG19 모델 불러오기 (EfficientNet과 유사한 방식 적용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ✅ 최신 PyTorch 권장 방식으로 사전 학습된 가중치 로드
model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)  # ✅ 최신 방식 사용
# ✅ 마지막 분류기 수정 (EfficientNet처럼 사용)
model.classifier[6] = nn.Linear(4096, num_classes)  # 출력층을 6개 클래스로 변경
model = model.to(device)

print("Using device:", device)

# ✅ 손실 함수 및 최적화 함수 정의 (EfficientNet과 동일한 방식 적용)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, test_loss):
        if self.best_loss is None or test_loss < self.best_loss - self.delta:
            self.best_loss = test_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, test_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
early_stopping = EarlyStopping(patience=10)

for epoch in range(epochs):
    print(f"\n[Epoch {epoch+1}/{epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "./best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Using device: cuda

[Epoch 1/50]


Train Loss: 1.9635 | Train Acc: 0.2076 | Test Loss: 1.7569 | Test Acc: 0.2825
  [*] Best model saved.

[Epoch 2/50]


Train Loss: 1.7976 | Train Acc: 0.2599 | Test Loss: 1.7555 | Test Acc: 0.2316

[Epoch 3/50]


Train Loss: 1.7284 | Train Acc: 0.2571 | Test Loss: 1.7110 | Test Acc: 0.1977

[Epoch 4/50]


Train Loss: 1.6853 | Train Acc: 0.2867 | Test Loss: 1.7171 | Test Acc: 0.2655

[Epoch 5/50]


...

Train Loss: 1.7398 | Train Acc: 0.2062 | Test Loss: 1.6991 | Test Acc: 0.2034

[Epoch 16/50]


Train Loss: 1.7423 | Train Acc: 0.2260 | Test Loss: 1.7003 | Test Acc: 0.2034

[Epoch 17/50]


Train Loss: 1.7455 | Train Acc: 0.2274 | Test Loss: 1.7023 | Test Acc: 0.2316

[Epoch 18/50]


Train Loss: 1.7444 | Train Acc: 0.2429 | Test Loss: 1.6979 | Test Acc: 0.2034

[Epoch 19/50]


Train Loss: 1.7501 | Train Acc: 0.2020 | Test Loss: 1.7066 | Test Acc: 0.2034
Early Stopping Triggered!

Best Accuracy: 0.2825


## efficientnet+vgg19

In [9]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Hybrid Model (EfficientNet + VGG19)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()
        
        # ✅ VGG19 사용
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        # Freeze EfficientNet의 초기 블록: Freeze blocks 0~2, Unfreeze 나머지
        for name, param in self.efficientnet.named_parameters():
            if "blocks.0" in name or "blocks.1" in name or "blocks.2" in name:
                param.requires_grad = False
            else:
                param.requires_grad = True

        # VGG 일부 Freeze (초반부)
        for name, param in self.vgg.named_parameters():
            if "features.0" in name or "features.1" in name or "features.2" in name or "features.3" in name or "features.4" in name or "features.5" in name:
                param.requires_grad = False
            else:
                param.requires_grad = True

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19 출력 채널

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.vgg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Linear(efficientnet_out_features + vgg_out_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = eff_features.view(eff_features.size(0), -1)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = vgg_features.view(vgg_features.size(0), -1)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)
        return self.classifier(fused_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.1377 | Train Acc: 0.6568 | Val Loss: 1.0561 | Val Acc: 0.6893
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.6675 | Train Acc: 0.9181 | Val Loss: 1.1404 | Val Acc: 0.7514
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.5639 | Train Acc: 0.9718 | Val Loss: 0.8426 | Val Acc: 0.8192
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.5174 | Train Acc: 0.9901 | Val Loss: 0.7389 | Val Acc: 0.8927
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.5311 | Train Acc: 0.9831 | Val Loss: 0.7890 | Val Acc: 0.8418

[Epoch 13/100]


Train Loss: 0.5212 | Train Acc: 0.9831 | Val Loss: 0.7529 | Val Acc: 0.8927

[Epoch 14/100]


Train Loss: 0.5353 | Train Acc: 0.9845 | Val Loss: 0.7710 | Val Acc: 0.8588

[Epoch 15/100]


Train Loss: 0.5426 | Train Acc: 0.9760 | Val Loss: 0.7693 | Val Acc: 0.8531

[Epoch 16/100]


Train Loss: 0.5271 | Train Acc: 0.9816 | Val Loss: 0.7400 | Val Acc: 0.8701
Early Stopping Triggered!

Best Accuracy: 0.8927


In [10]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Hybrid Model (EfficientNet + VGG19)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()
        
        # ✅ VGG19 사용
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19 출력 채널

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.vgg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Linear(efficientnet_out_features + vgg_out_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = eff_features.view(eff_features.size(0), -1)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = vgg_features.view(vgg_features.size(0), -1)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)
        return self.classifier(fused_features)

# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")


Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.2889 | Train Acc: 0.5847 | Val Loss: 1.4233 | Val Acc: 0.6271
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.7905 | Train Acc: 0.8602 | Val Loss: 1.0956 | Val Acc: 0.7345
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.6014 | Train Acc: 0.9421 | Val Loss: 0.9244 | Val Acc: 0.7514
  [*] Best model saved.

[Epoch 4/100]


Train Loss: 0.5811 | Train Acc: 0.9492 | Val Loss: 1.0155 | Val Acc: 0.7797
  [*] Best model saved.

[Epoch 5/100]


...

Train Loss: 0.5647 | Train Acc: 0.9590 | Val Loss: 0.8665 | Val Acc: 0.8362

[Epoch 15/100]


Train Loss: 0.5904 | Train Acc: 0.9407 | Val Loss: 0.9559 | Val Acc: 0.8023

[Epoch 16/100]


Train Loss: 0.5506 | Train Acc: 0.9619 | Val Loss: 0.8577 | Val Acc: 0.8362

[Epoch 17/100]


Train Loss: 0.5758 | Train Acc: 0.9492 | Val Loss: 0.9223 | Val Acc: 0.8192

[Epoch 18/100]


Train Loss: 0.6275 | Train Acc: 0.9251 | Val Loss: 1.4101 | Val Acc: 0.6723
Early Stopping Triggered!

Best Accuracy: 0.8927


In [14]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# ✅ SEED 설정 (재현성 확보)
seed = 2021
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ✅ EarlyStopping
class EarlyStopping:
    def __init__(self, patience=10, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ✅ Residual Block 정의
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return nn.ReLU()(x)

# ✅ ASPP Block 정의
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels=256):
        super(ASPP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.conv1_for_global = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv1x1_out = nn.Conv2d(out_channels * 5, out_channels, kernel_size=1)

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv3_1_out = self.conv3_1(x)
        conv3_6_out = self.conv3_6(x)
        conv3_12_out = self.conv3_12(x)

        global_avg = self.global_avg_pool(x)
        global_avg = self.conv1_for_global(global_avg)
        global_avg = nn.functional.interpolate(global_avg, size=conv1_out.shape[2:], mode='bilinear', align_corners=True)

        out = torch.cat([conv1_out, conv3_1_out, conv3_6_out, conv3_12_out, global_avg], dim=1)
        return self.conv1x1_out(out)

# ✅ Hybrid Model (EfficientNet + VGG19 + Residual + ASPP)
class HybridModel(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridModel, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.efficientnet._fc = nn.Identity()
        # ✅ VGG19 사용
        self.vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
        self.vgg.classifier = nn.Identity()

        # Freeze EfficientNet의 초기 블록: Freeze blocks 0~2, Unfreeze 나머지
        for name, param in self.efficientnet.named_parameters():
            if "blocks.0" in name or "blocks.1" in name or "blocks.2" in name:
                param.requires_grad = False
            else:
                param.requires_grad = True

        # VGG 일부 Freeze (초반부)
        for name, param in self.vgg.named_parameters():
            if "features.0" in name or "features.1" in name or "features.2" in name or "features.3" in name or "features.4" in name or "features.5" in name:
                param.requires_grad = False
            else:
                param.requires_grad = True

        efficientnet_out_features = self.efficientnet._conv_head.out_channels  # 1280
        vgg_out_features = 512  # VGG19 출력 채널

        self.efficientnet_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_pool = nn.AdaptiveAvgPool2d((7, 7))

        self.eff_conv1x1 = nn.Conv2d(efficientnet_out_features, 1024, kernel_size=1)
        self.vgg_conv1x1 = nn.Conv2d(vgg_out_features, 1024, kernel_size=1)

        self.res_block = ResidualBlock(2048, 2048)
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet.extract_features(x)
        eff_features = self.efficientnet_pool(eff_features)
        eff_features = self.eff_conv1x1(eff_features)

        vgg_features = self.vgg.features(x)
        vgg_features = self.vgg_pool(vgg_features)
        vgg_features = self.vgg_conv1x1(vgg_features)

        fused_features = torch.cat([eff_features, vgg_features], dim=1)  # (batch, 2048, 7, 7)
        fused_features = self.res_block(fused_features)
        aspp_features = self.aspp(fused_features)
        gap_features = self.global_avg_pool(aspp_features)
        flattened_features = gap_features.view(gap_features.size(0), -1)
        return self.classifier(flattened_features)



# ==============================
# ✅ 데이터 로딩
# ==============================
image_size = 224
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_path = "C:/Users/IIALAB/Desktop/kdm/solar/kaggle/input/solar-panel-images/Faulty_solar_panel"

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ==============================
# ✅ 모델, 손실 함수, 최적화기 설정
# ==============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=6).to(device)  # 모델을 먼저 생성해야 함!

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# ✅ Early Stopping 적용
early_stopping = EarlyStopping(patience=10)

# ✅ 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 평가 함수
def evaluate(model, val_loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

# ✅ 학습 루프
best_acc = 0.0
num_epochs = 100

for epoch in range(num_epochs):
    print(f"\n[Epoch {epoch+1}/{num_epochs}]")
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        print("  [*] Best model saved.")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early Stopping Triggered!")
        break

print(f"\nBest Accuracy: {best_acc:.4f}")



Loaded pretrained weights for efficientnet-b0

[Epoch 1/100]


Train Loss: 1.3824 | Train Acc: 0.5480 | Val Loss: 2.3458 | Val Acc: 0.5254
  [*] Best model saved.

[Epoch 2/100]


Train Loss: 0.8542 | Train Acc: 0.8150 | Val Loss: 0.8132 | Val Acc: 0.8531
  [*] Best model saved.

[Epoch 3/100]


Train Loss: 0.6719 | Train Acc: 0.8983 | Val Loss: 0.9239 | Val Acc: 0.8023

[Epoch 4/100]


Train Loss: 0.6610 | Train Acc: 0.9153 | Val Loss: 1.1160 | Val Acc: 0.7175

[Epoch 5/100]


...

Train Loss: 0.5348 | Train Acc: 0.9718 | Val Loss: 0.8219 | Val Acc: 0.8305

[Epoch 22/100]


Train Loss: 0.5211 | Train Acc: 0.9788 | Val Loss: 0.8292 | Val Acc: 0.8418

[Epoch 23/100]


Train Loss: 0.5081 | Train Acc: 0.9859 | Val Loss: 0.7622 | Val Acc: 0.8475

[Epoch 24/100]


Train Loss: 0.5196 | Train Acc: 0.9816 | Val Loss: 0.8022 | Val Acc: 0.8588

[Epoch 25/100]


Train Loss: 0.4883 | Train Acc: 0.9915 | Val Loss: 0.7800 | Val Acc: 0.8531
Early Stopping Triggered!

Best Accuracy: 0.9379
